# Liquid Argon model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
import simtk.unit as unit
from simtk import openmm as mm
from simtk.openmm import app
import skopt as skopt
from tqdm import tqdm

In [2]:
n_particles = 1000
reduced_density = 0.75
mass = 39.948 * unit.amu
sigma = 3.404 * unit.angstroms
epsilon = 0.238 * unit.kilocalories_per_mole
charge = 0.0 * unit.elementary_charge

temperature = 300.0 * unit.kelvin

integration_timestep = 2.0 * unit.femtoseconds
collisions_rate = 1.0 / unit.picoseconds

equilibration_time = 1.0 * unit.nanoseconds

production_time = 5.0 * unit.nanoseconds
saving_time = 50.0 * unit.picoseconds

The Van der Waals radius of Argon is 1.88 angstroms.

In [3]:
radius = 2.0**(-5/6) * sigma
print(radius)

1.9104304062225528 A


In [4]:
volume_particles = n_particles * sigma**3
volume = volume_particles/reduced_density
l_box = volume**(1/3)

In [5]:
system = mm.System()

In [6]:
v1 = np.zeros(3) * unit.angstroms
v2 = np.zeros(3) * unit.angstroms
v3 = np.zeros(3) * unit.angstroms

v1[0] = l_box
v2[1] = l_box
v3[2] = l_box

system.setDefaultPeriodicBoxVectors(v1, v2, v3)

In [7]:
non_bonded_force = mm.NonbondedForce()
non_bonded_force.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
non_bonded_force.setCutoffDistance(3.0*sigma)
non_bonded_force.setUseSwitchingFunction(True)
non_bonded_force.setSwitchingDistance(2.0*sigma)
non_bonded_force.setUseDispersionCorrection(True)

In [8]:
for _ in range(n_particles):
    system.addParticle(mass)
    non_bonded_force.addParticle(charge, sigma, epsilon)

In [9]:
_ = system.addForce(non_bonded_force)

In [10]:
integrator = mm.LangevinIntegrator(temperature, collisions_rate, integration_timestep)
platform = mm.Platform.getPlatformByName('CUDA')
context = mm.Context(system, integrator, platform)

In [11]:
space = skopt.Space([[0.0, l_box._value], [0.0, l_box._value], [0.0, l_box._value]])
grid_generator = skopt.sampler.Grid(use_full_layout=False)
initial_positions = grid_generator.generate(space.dimensions, n_particles)
initial_positions = np.array(initial_positions)*unit.angstroms

In [12]:
context.setPositions(initial_positions)
context.setVelocitiesToTemperature(temperature)

In [13]:
state = context.getState(getPositions=True)
positions = state.getPositions(asNumpy=True)

In [14]:
positions.min()

Quantity(value=0.11353293061256409, unit=nanometer)

In [ ]:
state=context.getState(getEnergy=True)
print("Before minimization: {}".format(state.getPotentialEnergy()))
mm.LocalEnergyMinimizer_minimize(context)
state=context.getState(getEnergy=True)
print("After minimization: {}".format(state.getPotentialEnergy()))

In [15]:
state.getPeriodicBoxVectors()

Quantity(value=[Vec3(x=3.7465867850791024, y=0.0, z=0.0), Vec3(x=0.0, y=3.7465867850791024, z=0.0), Vec3(x=0.0, y=0.0, z=3.7465867850791024)], unit=nanometer)

In [ ]:
state = context.getState(getPositions=True)
positions = state.getPositions(asNumpy=True)

In [ ]:
positions.max()